In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
raw_data = pd.read_csv("대파_1y_일별_정산_경락가격.csv")

/Users/sungjune/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


|       DELNG_DE       |       경락일       |
|:--------------------:|:------------------:|
| PBLMNG_WHSAL_MRKT_NM |   공영 도매시장명  |
| PBLMNG_WHSAL_MRKT_CD | 공영 도매시장 코드 |
|        CPR_NM        |       법인명       |
|        CPR_CD        |      법인코드      |
|        RISENO        |      원표번호      |
|        BIDTIME       |      경매시간      |
|      AUC_SE_CODE     |    경매구분코드    |
|       AUC_SE_NM      |      경매구분      |
|         ORGNO        |      일련번호      |
|       PRDLST_NM      |       품목명       |
|       PRDLST_CD      |      품목코드      |
|       SPCIES_NM      |       품종명       |
|       SPCIES_CD      |      품종코드      |
|   CPR_USE_PRDLST_NM  |    법인사용품명    |
|   CPR_USE_SPCIES_NM  |   법인사용품종명   |
|         PRICE        |        가격        |
|    DELNGBUNDLE_QY    |      거래단량      |
|        STNDRD        |        규격        |
|       STNDRD_CD      |      규격코드      |
|         GRAD         |        등급        |
|        GRAD_CD       |      등급코드      |
|    SHIPMNT_SE_CODE   |    출하구분코드    |
|     SHIPMNT_SE_NM    |     출하구분명     |
|       SANJI_CD       |      산지코드      |
|       SANJI_NM       |       산지명       |
|   CPR_USE_SANJI_CD   |    법인산지코드    |
|   CPR_USE_SANJI_NM   |     법인산지명     |
|       DELNG_QY       |       거래량       |

In [8]:
data1 = raw_data.copy()

# 산지가 결측이면 법인산지로 대치함
data1.loc[data1["SANJI_NM"].isna(), "SANJI_NM"] = \
    data1.loc[data1["SANJI_NM"].isna(), "CPR_USE_SANJI_NM"]

# 산지를 찾을 수 없으면 삭제함(2021년 7월 약 1.84%)
na_n = len(data1.loc[data1["SANJI_NM"].isna()])
total_n = len(data1)
print(f"""결측치 비율: {format(na_n / total_n * 100, ".2f")}%""")
print(f"""결측치 개수: {na_n}개""")
drop_index = data1.loc[data1["SANJI_NM"].isna(), "SANJI_NM"].index
data1.drop(drop_index, inplace=True)

# 대파(일반)만 가져옴
temp = len(data1.loc[data1["SPCIES_NM"] == "대파(일반)"]) / len(data1) * 100
print("대파(일반)의 비율: {0:.2f}%".format(temp))
# print(data1["SPCIES_NM"].value_counts())
data2 = data1.loc[data1["SPCIES_NM"] == "대파(일반)"]

# ["대파(일반)", "대파", "대파(단)", "파(대파)"]만 가져옴
data3 = pd.DataFrame()
for spcies in ["대파(일반)", "대파", "대파(단)", "파(대파)"]:
    try:
        temp = data2.loc[data2["CPR_USE_SPCIES_NM"] == spcies]
        data3 = pd.concat([data3, temp])
    except:
        print(f"CPR_USE_SPCIES_NM {spcies} 없음")

# 수입 삭제
drop_index = data3.loc[data3["SANJI_NM"].str.contains("수입")].index
data3.drop(drop_index, inplace=True)

# Unique 값 체크
print("수입 삭제 전 n(산지):", data1["SANJI_NM"].nunique())
print("수입 삭제 전 n(산지):", data3["SANJI_NM"].nunique())

# 거래량을 kg 단위로 변경함
data4 = data3.copy()
data4["거래량/kg"] = data4["DELNG_QY"] * data4["DELNGBUNDLE_QY"]


# 8도 처리
# SANJI_NM 결측치 CPR_USE_SANJI_NM로 대체
# STNDRD kg 없으면 삭제

결측치 비율: 1.43%
결측치 개수: 4376개
대파(일반)의 비율: 94.88%
수입 삭제 전 n(산지): 933
수입 삭제 전 n(산지): 912


```
========================================================
전남 신안 => 신안
전남 진도 + 해남 => 진도
전남(신안, 진도, 해남 빼고) => 영광
전북 => 전주
부산 + 경남(김해 + 창원 + 울산...) => 부산
충북 => 청주
충남 => 아산
경북 + 대구 => 구미
강원 => 평창
서울 + 안양 + 구리 + 남양주 + 하남 + 광명 + 시흥 + 성남 => 구리
수원 + 안성 + 오산 + 평택 + 이천 + 여주 + 양평 + 용인 + 광주 + 화성 => 용인
고양 + 파주 + 양주 + 동두천 + 연천 + 포천 + 가평 + 의정부 + 경기 => 양주
인천 + 김포 + 부천 + 안산 + 군포 => 인천
========================================================
```

In [13]:
def check_si_gun_gu(text):
    """
    시/군/구가 포함되어 있는지 확인한다.
    """
    if (text[-1] == "시") | (text[-1] == "군") | (text[-1] == "구"):
        return True
    else:
        return False

def check_double(text):
    """
    광역자치단체 명이 중복되어 있는지 확인한다.
    """
    sido_dict = {"경기도": "경기",
                 "강원도": "강원",
                 "경상북도": "경북",
                 "경상남도": "경남",
                 "충청북도": "충북",
                 "충청남도": "충남",
                 "전라북도": "전북",
                 "전라남도": "전남",
                 "서울특별시": "서울",
                 "세종특별시": "세종",
                 "부산광역시": "부산",
                 "울산광역시": "울산",
                 "대구광역시": "대구",
                 "대전광역시": "대전",
                 "제주특별자치도": "제주",
                 "제주도": "제주",
                 "광주광역시": "광주",
                 "인천광역시": "인천",
                }
    temp_list = sido_dict.values()
    if text in temp_list:
        return True
    else:
        return False

def loca_name_change(data):
    """
    광역자치단체 명과 시군구 명을 통일한다.
    """
    sido_dict = {"경기도": "경기",
                 "강원도": "강원",
                 "경상북도": "경북",
                 "경상남도": "경남",
                 "충청북도": "충북",
                 "충청남도": "충남",
                 "전라북도": "전북",
                 "전라남도": "전남",
                 "서울특별시": "서울",
                 "세종특별시": "세종",
                 "부산광역시": "부산",
                 "울산광역시": "울산",
                 "대구광역시": "대구",
                 "대전광역시": "대전",
                 "제주특별자치도": "제주",
                 "제주도": "제주",
                 "광주광역시": "광주",
                 "인천광역시": "인천",
                 }

    temp = list(filter(None, data.split(" ")))
    
    # 광역시도명 짧게 변경
    if temp[0] in sido_dict.keys():
        temp[0] = sido_dict[temp[0]]
    
    # 시군구 짧게 변경
    if len(temp) >= 2:
        if check_double(temp[1]):
            temp[1] = temp[2]

        if check_si_gun_gu(temp[1]): 
            res = temp[0] + " " + temp[1][:-1] # 시군구 삭제하고 결합
        else:
            res = temp[0] + " " + temp[1]
    else:
        res = temp[0]
    
    return res    

In [14]:
data4.loc[:, "SANJI_NM"] = data4.loc[:, "SANJI_NM"].apply(loca_name_change)
# np.sort(data4.loc[:, "SANJI_NM"].unique())

In [26]:
def merge_loca(data):
    """
    사전에 정의한 지리적으로 근접한 지역을 합침.
    """

    def exception_process(text):
        """
        예외적인 지역 이름을 처리함.
        """
        # 리스트로 들어오면 문자열로 변경함
        text = ' '.join(text)
        exception_dict = {"전주": "전라남도 영광+",
                        }

        # 시군구가 포함되어 있으면 시군구로 변경함
        for area in area_dict.keys():
            if area in text:
                text = area_dict[area]
                # 종료
                return text

        # 예외 처리 항목에 포함되어 있으면 해당 값으로 변경함
        for area in exception_dict.keys():
            if area in text:
                text = exception_dict[area]
                return text
        
        # 광역지자체가 포함되어 있으면 광역지자체로 변경함
        for area in metro_area_dict.keys():
            if area in text:
                text = metro_area_dict[area]
                return text
        
        # 처리되지 않았으면 메세지 출력함 
        print('='*79)
        print(f"{text} 처리 필요함!!!".center(60, ' '))
        print('='*79)
        
        return text

    area_dict = {"신안": "전라남도 신안",
                 "진도": "전라남도 진도/해남",
                 "해남": "전라남도 진도/해남",
                 "안양": "경기도 동남권(서울, 구리+)",
                 "구리": "경기도 동남권(서울, 구리+)",
                 "남양주": "경기도 동남권(서울, 구리+)",
                 "하남": "경기도 동남권(서울, 구리+)",
                 "성남": "경기도 동남권(서울, 구리+)",
                 "광주": "경기도 동남권(서울, 구리+)",
                 "수원": "경기도 남부(용인+)",
                 "안성": "경기도 남부(용인+)",
                 "오산": "경기도 남부(용인+)",
                 "평택": "경기도 남부(용인+)",
                 "이천": "경기도 남부(용인+)",
                 "여주": "경기도 남부(용인+)",
                 "양평": "경기도 남부(용인+)",
                 "용인": "경기도 남부(용인+)",
                 "화성": "경기도 남부(용인+)",
                 "고양": "경기도 북부(양주+)",
                 "파주": "경기도 북부(양주+)",
                 "양주": "경기도 북부(양주+)",
                 "동두천": "경기도 북부(양주+)",
                 "연천": "경기도 북부(양주+)",
                 "포천": "경기도 북부(양주+)",
                 "가평": "경기도 북부(양주+)",
                 "의정부": "경기도 북부(양주+)",
                 "김포": "인천/경기 서남권(인천+)",
                 "부천": "인천/경기 서남권(인천+)",
                 "안산": "인천/경기 서남권(인천+)",
                 "군포": "인천/경기 서남권(인천+)",
                 "광명": "인천/경기 서남권(인천+)",
                 "시흥": "인천/경기 서남권(인천+)",
                 "평창": "강원도 평창+",
                }
    metro_area_dict = {"전남": "전라남도 영광+",
                       "전북": "전라북도 전주+",
                       "경남": "부산(경상남도+울산)",
                       "충북": "충청북도 청주+",
                       "충남": "충청남도 아산+",
                       "경북": "경상북도 구미+(+대구)",
                       "경기": "경기도 북부(양주+)",
                       "광주": "전라남도 영광+",
                       "부산": "부산(경상남도+울산)",
                       "울산": "부산(경상남도+울산)",
                       "대전": "충청북도 청주+",
                       "세종": "충청북도 청주+",
                       "대구": "경상북도 구미+(+대구)",
                       "서울": "경기도 동남권(서울, 구리+)",
                       "인천": "인천/경기 서남권(인천+)",
                       "강원": "강원도 평창+",
                      }

    temp = list(filter(None, data.split(" ")))
    
    if len(temp) >= 2:
        if temp[1] in area_dict.keys():
            res = area_dict[temp[1]]
        elif temp[0] in metro_area_dict.keys():
            res = metro_area_dict[temp[0]]
        else:
            res = exception_process(temp)
    else:
        if temp[0] in metro_area_dict.keys():
            res = metro_area_dict[temp[0]]
        else:
            res = exception_process(temp)

    return res

In [27]:
data5 = data4.copy()

# 제주도는 생산량이 미미하므로 제거함
drop_index = data5[data5["SANJI_NM"].str.contains("제주")].index
data5.drop(drop_index, inplace=True)
data5.loc[:, "SANJI_NM"] = data5.loc[:, "SANJI_NM"].apply(merge_loca)

In [31]:
# data5["SANJI_NM"].unique()
# data5["SANJI_NM"].value_counts()
print(data5["SANJI_NM"].nunique())

13


In [ ]:
###############################################################################
# 시군구 기준 거래량 상위 50개 지역
###############################################################################
group_temp = data4.loc[:, ["SANJI_NM", "거래량/kg"]].groupby("SANJI_NM").sum()
total_q = int(data4[["거래량/kg"]].sum().values)
group_temp["ratio"] = \
    group_temp["거래량/kg"].apply(lambda x: format(x / total_q * 100, ".2f"))

print('=' * 79)
print("시군구 기준 거래량 상위 50개 지역")
print('=' * 79)
print(group_temp.sort_values(by="거래량/kg", ascending=False).head(50))
print('=' * 79, end="\n\n\n")
###############################################################################
###############################################################################


###############################################################################
# 지역 기준 거래량 상위 지역
district = "경기"
###############################################################################
group_temp = group_temp.reset_index()

# print('=' * 79)
# print(f"{district} 기준 거래량 상위 50개 지역")
# print('=' * 79)
# print(group_temp.loc[group_temp.SANJI_NM.str.contains(district)]\
#     .sort_values(by="거래량/kg", ascending=False))
# print('=' * 79, end="\n\n\n")
###############################################################################
###############################################################################

###############################################################################
# 광역지자체 기준 거래량 상위 지역
###############################################################################
def pop_sido(data):
    temp = list(filter(None, data.split(" ")))
    res = temp[0]
    return res    

group_temp["SANJI_NM"] = group_temp["SANJI_NM"].apply(pop_sido)
group_temp = group_temp.groupby("SANJI_NM").sum()
group_temp["ratio"] = \
    group_temp["거래량/kg"].apply(lambda x: format(x / total_q * 100, ".2f"))

print('=' * 79)
print("광역 지자체 기준 거래량 상위 지역")
print('=' * 79)
print(group_temp.sort_values(by="거래량/kg", ascending=False))
print('=' * 79, end="\n\n\n")
###############################################################################
###############################################################################


시군구 기준 거래량 상위 50개 지역
              거래량/kg  ratio
SANJI_NM                   
전남 진도     9834753.40  10.57
전남 신안     8156342.60   8.76
충북 청주     4808949.20   5.17
부산        4518184.30   4.85
전북 전주     3243906.25   3.49
부산 강서     2851288.00   3.06
강원 평창     2831017.30   3.04
전북 완주     2706531.90   2.91
전남 영광     2552986.50   2.74
경기 구리     1960767.50   2.11
충남 아산     1845426.60   1.98
경남 김해     1742538.90   1.87
전북 익산     1687667.63   1.81
전남 해남     1686466.00   1.81
경기 수원     1630302.50   1.75
충북 청원     1628109.80   1.75
충남 서산     1601998.00   1.72
인천 계양     1599214.00   1.72
경기 고양     1555234.00   1.67
울산 남      1336876.60   1.44
전북 부안     1325716.00   1.42
인천        1245474.00   1.34
경기 남양주    1181464.40   1.27
경기 포천     1014367.10   1.09
경남 창원      979105.02   1.05
경기 안성      970066.80   1.04
경기 양주      968869.00   1.04
충남 천안      942866.50   1.01
경기 이천      914063.00   0.98
대전 유성      758074.00   0.81
부산 중       664498.50   0.71
전북 고창      647627.40   0.70
경남 밀양      640051.90   0.69